### Section 9 - Self Organizing Maps (SOM)
Self Organizing Maps are an unsupervised deep learning algorithm used for feature detection. Specifically, SOMs are used for reducing dimensionality. They take a multidimensional dataset and reduce it to a 2 dimensional representation of your dataset. 
![](./images/17.jpg "")

##### How Self Organizing Maps work
The are similar to the input / output architecture of Neural Networks except the weights are not actual weights that are multiplied but coordinate representation in 2 dimensions; characteristics of the node itself. Inputs are values between 0 and 1 so must be standardized or normalized. 
![](./images/18.jpg "") 
Whichever node has the smallest distance represents the best matching unit(BMU). After finding the BMU, the SOM is going to update the "weights" so that BMU so it is even closer to the row that was input in. Since you move the BMU, you will move the entire node set closer to that row of value 
![](./images/19.jpg "") 
Self Organizing Maps retain the topology of the input set and often times reveal correlations that are not easily identified. Since they are unsupervised, we do not need to worry about preliminary relations

### Steps of Self Organizing Maps
 1. Start with dataset composed of n_features independent variables
 2. Create a grid composed of nodes, each on have a weight vector of n_features elements
 3. Randonly intialize the valuies of the weight vectors to small numbers close to 0
 4. Select one random observation point from the dataset
 5. Compute Euclidean distances from this point to the different neurons in the network
 6. Select the neuron that has the minimum distance to the point. This neuron is called the winning node
 7. Update the weights of the winning node to move it closer to the point
 8. Using a gaussian neighboorhood function of the mean of the winning mode, also update the weights of the winning node neighbors to move them closer to the point. The neighboorhood radius is the sigma ion the gaussian function
 9. Repeat s 1 to 5 and update the weights after each observation(reinforcement learning) or after a batch of observations(bach learning) until the network converges to a point where the neighborhood stops decreasing. 

![](./images/20.jpg "") 

##### How we will use SOMs 
For our case, we hope to map our feature set where the outliers are also vechiles that are bad buys. We will use a dimensionailty reduction to make mapping to the SOM easier. 


In [1]:
import numpy as np   #Mathematics library
import matplotlib.pyplot as plt # for plotting
import pandas as pd  #manage datasets
import seaborn as sea
import scipy.stats as stats
import sklearn

In [2]:
df = pd.read_csv('ImputeMissing.csv')
df=df.drop('Unnamed: 0',axis=1)

In [ ]:
#Rearrange Column Order
cols = df.columns.tolist()
cols = cols[1:2]+cols[0:1] + cols[2:]
df = df[cols]


In [3]:
###Convert 15 Categorical to Dummies
df=pd.get_dummies(df,columns=['Auction','Make','Model','Trim','Color','Transmission','WheelType','Nationality','Size','TopThreeAmericanName','PRIMEUNIT','AUCGUART','VNZIP1','VNST','IsOnlineSale'],drop_first=True)


In [4]:
# Importing the dataset
dataset = df
X = dataset.iloc[:, 2:].values
y = dataset.iloc[:, 1].values

In [6]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
X = sc.fit_transform(X)

In [ ]:
# Training the SOM
from minisom import MiniSom
som = MiniSom(x = 10, y = 10, input_len = 14, sigma = 1.0, learning_rate = 0.5)
#input_len = is variables, but include ID so you can predict lemon
# 10 x 10 arbitraily, choose bigger for our dataset
#higher learning rate, faster the converges
som.random_weights_init(X)
som.train_random(data = X, num_iteration = 10)

In [ ]:
# Visualizing the results
from pylab import bone, pcolor, colorbar, plot, show
bone()
pcolor(som.distance_map().T)
colorbar()
markers = ['o', 's']
colors = ['r', 'g']
for i, x in enumerate(X):
    w = som.winner(x)
    plot(w[0] + 0.5,
         w[1] + 0.5,
         markers[y[i]],
         markeredgecolor = colors[y[i]],
         markerfacecolor = 'None',
         markersize = 10,
         markeredgewidth = 2)
show()

In [ ]:
# Finding the frauds
mappings = som.win_map(X)
frauds = np.concatenate((mappings[(9,2)], mappings[(8,4)],mappings[(2,6)],mappings[(8,5)]), axis = 0)
frauds = sc.inverse_transform(frauds)


In [ ]:
# Creating the dependent variable
is_fraud = np.zeros(len(dataset))
for i in range(len(dataset)):
    if dataset.iloc[i,0] in frauds:
        is_fraud[i] = 1

### Testing Model Accuracy

In [12]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, is_fraud)
cm

In [ ]:
# Accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y, is_fraud)

In [13]:
# Precision 
from sklearn.metrics import precision_score
precision_score(y, is_fraud)


NameError: name 'precision_score' is not defined

In [ ]:
# Recall
from sklearn.metrics import recall_score
recall_score(y, is_fraud)


In [ ]:
# F1 score
from sklearn.metrics import f1_score
f1_score(y, is_fraud)


In [ ]:
# Cohen's kappa
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(y, is_fraud)


In [ ]:
accuracies.std()